<font color='green'> 
**Youtube - Aladdin Persson Kanalı - TensorFlow 2.0 Beginner Tutorials serisi**
    
TensorFlow Tutorial 10 - Saving and Loading Models - Aladdin Persson anlattı.
</font>

**Video**: [TensorFlow Tutorial 10 - Saving and Loading Models](https://www.youtube.com/watch?v=idus3KO6Wic&list=PLhhyoLH6IjfxVOdVC1P1L5z5azs0XjMsb&index=10)

### İçindekiler

**Loading Dataset**

**Preprocessing Dataset**

**Creating Models**
* Using Sequential API
* Using Functional API
* Using Subclassing

**Save and Load Model Weights**
* Save Model Weights
* Load Model Weights 

**Save and Load Entire Model (Serializing Model)**
* Save Entire Model
* Load Entire Model

### <font color="blue"> Giriş</font>

Bu notebookta amaç modelin weightlerini ve tüm modeli nasıl save ve load edeceğimizi öğreneceğiz.

#### **1. How to save and load model weights** 

#### **2. Save and load entire model (serializing model)**
Tüm modeli kaydedip yüklerken, bir data structure olarak kaydedilecek ve bu, modelin tensorflow javascript, tensorflow lite gibi farklı tensorflow frameworklerine yüklenebileceği anlamına geliyor. Yani örneğin PC'nizde bir modeli eğitebilir ve diyelim ki onu productiona almak ve bir uygulama yapmak istiyorsunuz, daha önce eğittiğiniz modeli yükleyebilirsiniz ve herhangi bir dönüştürme yapmanıza gerek olmaz.

Tüm modeli yüklediğimizde:
* Weightler kaydedilecek. 
* Model architecture'ı (model mimarisi) kaydedilecek. Bu şekilde modelin koduna ihtiyacımız kalmayacak.
* Training configurationı da (eğitim yapılandırmasını) da kaydedecek, böylece model.compile() dosyasına göndereceğiniz şey bu olacaktır.
* Optimizer ve state'leri kaydedecek. So for example if you're using the Adam optimizer it's gonna keep track of the exponential weighted averages and that's internal to the optimizer. Örneğin, yalnızca weightleri kaydediyorsanız, modelden weightleri her yüklediğinizde optimize edici durumlar sıfırlanacaktır.

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### 1. Loading Dataset

In [2]:
from tensorflow.keras.datasets import mnist

In [3]:
(x_train, y_train),(x_test, y_test) = mnist.load_data()

In [4]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


### 2. Preprocessing Dataset

In [5]:
x_train = x_train.reshape(-1, 28 * 28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28*28).astype("float32") / 255.0

### 3. Creating Models

#### <font color="green">3 farklı model yazdık. </font>

 <font color="black">**İlkinde Sequential API kullandık.** </font>

In [6]:
model1 = keras.Sequential(
    [
     layers.Dense(64, activation='relu'),
     layers.Dense(10)
    ]
)

<font color="black">**İkincisinde aynı modeli Functional API kullanarak yazdık.** </font>

In [7]:
inputs = keras.Input(784)
x = layers.Dense(64, activation='relu')(inputs)
outputs = layers.Dense(10)(x)
model2 = keras.Model(inputs=inputs, outputs=outputs)

<font color="black">**Üçüncüsünde modelimizi subclassing kullanarak yazdık.** </font>

In [8]:
class MyModel(keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = layers.Dense(64, activation='relu')
        self.dense2 = layers.Dense(10)

    def call(self, input_tensor):
        x = tf.nn.relu(self.dense1(input_tensor))
        return self.dense2(x)

model3 = MyModel()

### 4. Save and Load Model Weights 

#### <font color="blue">Save Model Weights </font>

In [9]:
model = model1 #İlk modeli eğitip kaydedeceğiz. 

In [10]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [11]:
model.fit(x_train, y_train, batch_size=32, epochs=2, verbose=2)

Epoch 1/2
1875/1875 - 1s - loss: 0.2974 - accuracy: 0.9167
Epoch 2/2
1875/1875 - 2s - loss: 0.1420 - accuracy: 0.9581


In [12]:
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

313/313 - 0s - loss: 0.1237 - accuracy: 0.9632


[0.12372887134552002, 0.9631999731063843]

<font color="green">**Weightleri save etmek için `model.save_weights(..)` diyoruz.**</font>

In [13]:
model.save_weights('10. TensorFlow Tutorial 10 - Saving and Loading Models/saved_model/') 

* `model.save_weights('saved_model/')`: bir klasör adı yazıyoruz parantez içine. Notebookun bulunduğu yerde bu adda bir klasör açıyor. Scripti çalıştırdığımız yere baktığımızda saved_model adında bir klasör göreceğiz ve weightlerin kaydedildiği dosyalar bu klasörün içinde yer alıyor. (Bilgisayarda çalıştırınca oluyor bu işlem colabde olmuyor)

* "save_format" argümanı ile `model.save_weights('saved_model/', save_format='h5')` ile farklı formatlarda kaydedebiliriz weightleri. If we specify “.h5”, the model will be saved in hdf5 format; if no extension is specified, the model will be saved in TensorFlow default format.

#### <font color="blue">Load Model Weights </font>

**Baştan açtım notebooku diyelim ki bu şekilde çalıştırmam gerekiyor:**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

<font color="green">**Daha önce kaydettiğimiz weightleri load etmek için `model.load_weights(..)` diyoruz.**</font>

In [2]:
model.load_weights('10. TensorFlow Tutorial 10 - Saving and Loading Models/saved_model/') 

NameError: name 'model' is not defined

Modeli kaydetmemiştik sadece weightleri kaydetmiştik. Bu yüzden "name 'model' is not defined" hatası verdi. Model tanımlamamız lazım öncesinde.

In [3]:
model1 = keras.Sequential(
    [
     layers.Dense(64, activation='relu'),
     layers.Dense(10)
    ]
)

In [4]:
model = model1

In [5]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [6]:
model.load_weights('10. TensorFlow Tutorial 10 - Saving and Loading Models/saved_model/') 

##### x_test, y_test vermemiz gerekir öncesinde baştan çalıştırıyorsak.

In [7]:
from tensorflow.keras.datasets import mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_test = x_test.reshape(-1, 28*28).astype("float32") / 255.0

##### ---

In [8]:
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

313/313 - 0s - loss: 0.1237 - accuracy: 0.9632


[0.12372887134552002, 0.9631999731063843]

####  <font color="purple">Dikkat!</font>
<font color="black">**Burada dikkat edilmesi gereken nokta farklı uygulamalar için weightleri takas edemeyiz. Mesela bu kaydettiğimiz weightleri model2 için kullanamayız uyumlu olmadığıyla ilgili hata alırız. Buradaki genel kural modeli save ettiğin şekilde load etmek. Yani mesela sequential olarak modeli create edip save ettiysen load ederken de bu formda bir modele load etmen gerekiyor.**</font>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
inputs = keras.Input(784)
x = layers.Dense(64, activation='relu')(inputs)
outputs = layers.Dense(10)(x)
model2 = keras.Model(inputs=inputs, outputs=outputs)

In [3]:
model = model2

In [4]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [5]:
model.load_weights('10. TensorFlow Tutorial 10 - Saving and Loading Models/saved_model/') 


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000001CC03E03448> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x000001CC03DDDC48>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x000001CC03E03748> and <tensorflow.python.keras.layers.core.Dense object at 0x000001CC03E03448>).


In [6]:
from tensorflow.keras.datasets import mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_test = x_test.reshape(-1, 28*28).astype("float32") / 255.0

In [7]:
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

313/313 - 0s - loss: 0.1237 - accuracy: 0.9632


[0.12372887134552002, 0.9631999731063843]

### 5. Save and Load Entire Model (Serializing Model)

- Save the weights
- Model arhitecture
- Training Configuration (model.compile())
- Optimizer and states

#### <font color="blue">Save Entire Model</font>

<font color="green">**Modeli save etmek için `model.save(...)` diyoruz.**</font>

In [11]:
model = model2

In [12]:
model.save('10. TensorFlow Tutorial 10 - Saving and Loading Models/complete_saved_model/')

INFO:tensorflow:Assets written to: 10. TensorFlow Tutorial 10 - Saving and Loading Models/complete_saved_model/assets


#### <font color="blue">Load Entire Model</font>

<font color="green">**Modeli load etmek için `keras.models.load_model(...)` diyoruz.**</font>

In [2]:
model = keras.models.load_model('10. TensorFlow Tutorial 10 - Saving and Loading Models/complete_saved_model/')

*WARNING:tensorflow: "No training configuration found in save file, so the model was not compiled. Compile it manually." uyarısını göz ardı edebiliriz.*

In [3]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________


Pretrained modeli nasıl kullanacağımızı "11. TensorFlow Tutorial 11 - Transfer Learning, Fine Tuning and TensorFlow Hub" notebookunda gördük.